# Keras
## Introducción
Keras es un API de alto nivel para redes neuronales, escrita en Python que es capaz de correr sobre TensorFlow, CNTK o Theano. Fue desarrollada con énfasis en habilitar experimentación rápida. Es recomendable el uso de Keras cuando se requiere de una libería de Deep Learning que:
- Permita el desarrollo de prototipos fácil y rápido (por medio de facilidad de uso, modularidad y extensibilidad).
- Soporte redes neuronales convolucionales (CNN) y redes neuronales recurrentes (RNN), así como combinaciones de ambas.
- Corra de forma natural en CPUs y GPUs.

Keras es compatible con Python 2.7-3.6.

[keras.io](https://keras.io) es el sitio principal del proyecto.

## Backend Keras y tf.keras
Se recomienda usar `tf.keras` a usuarios que usen Keras multi-backend con TensorFlow en TensorFlow 2.0 ya que `tf.keras` tiene mejor integración con las características de TensorFlow 2.0 (eager execution, soporte a distribución).

## Principios
- **Amigable**. El API está diseñado para personas, es consistente, reduce el número de pasos para casos de uso comunes. 
- **Modular**. Un modelo de deep learning consta de una secuencia o un grafo de módulos configurables que pueden ser conectados  con la menor cantidad de restricciones. Las capas de las redes neuronales, las funciones de costo, los optimizadores, esqueasm de inicialización, funciones de activación y esquemas de regularización son módulos independientes que pueden combinarse para crear nuevos modelos.
- **Extensible**. Es posible añadir módulos nuevos
- **Trabaja con Python**. No requiere de archivos de configuración de modelos. Los modelos se describen en Python, que es compacto, fácil de depurar y facilita la extensibilidad.

## Estructuras de datos
La estructura central de Keras es un modelo, que representa una forma de organizar capas. `Sequential` representa al modelo más simple, una pila linear de capas. El **Functional API** permite crear arquitecturas más complejas por medio de un grafo arbitrario de capas.

## Clasificación de dígitos en Keras


Importar datos

In [9]:
import numpy as np
import mnist
import keras

train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


Preprocesar los datos

In [10]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(test_images.shape)
print(test_labels.shape)

(10000, 784)
(10000,)


Construir el modelo

`model = Sequential([  #layers...])`

Para el caso de una red neuronal feed forward solo se requiere de la capa [Dense](keras.io/layers/core/#dense), que representa una capa completamente conectada.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])
